In [22]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

### Quickstart

In [23]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [24]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi!I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f4703499-eaa1-42e8-82b1-3d10af70ca8c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21})

In [25]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not have access to that information. Could you please provide me with your name?", response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 12, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d3097b81-f3e3-43cb-95f8-5250a9e76fdf-0', usage_metadata={'input_tokens': 12, 'output_tokens': 22, 'total_tokens': 34})

In [26]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="Hello Bob! How can I assist you today?"),
    HumanMessage(content="What's my name?")
])

AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6fadc920-cfa8-400d-b901-64319b94bdf0-0', usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40})

### Message History

In [27]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [28]:
config = {"configurable": {"session_id": "abc2"}}

In [29]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config
)
response.content

'Nice to meet you, Bob! How can I assist you today?'

In [30]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob.'

In [31]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I am an AI assistant and do not have access to personal information. Can I assist you with anything else?"

In [32]:
store

{'abc2': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Bob"), AIMessage(content='Nice to meet you, Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 12, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f122bd20-da9c-47e6-9637-6ecae03ff4d7-0', usage_metadata={'input_tokens': 12, 'output_tokens': 14, 'total_tokens': 26}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 39, 'total_tokens': 44}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e8b9da7b-db3d-4e65-84d3-0ea0ae29eadc-0', usage_metadata={'input_tokens': 39, 'output_tokens': 5, 'total_tokens': 44})]),
 'abc3': InMemoryChatMessageHistory(messages=[HumanMessage(content="What's my name?"), AI

### Prompt templates

In [33]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

promt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. Answer all questions to the best of your ability."
    ),
    MessagesPlaceholder(variable_name="messages")
])

chain = promt | model

In [34]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})
response.content

'Hello, Bob! How can I assist you today?'

In [35]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [36]:
config = {"configurable": {"session_id": "abc5"}}

In [37]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hello, Jim! How can I assist you today?'

In [38]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim. How can I assist you further, Jim?'

In [39]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [40]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], 
     "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

In [41]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [42]:
config = {"configurable": {"session_id": "abc11"}}

In [43]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], 
     "language": "Spanish"},
    config=config,
)

response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

### Managing Conversation History

In [44]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [45]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don't have access to that information."

In [46]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what 2 + 2 is.'

In [48]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, but I don't have access to your name. How may I assist you today?"

In [49]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked a math problem yet. Feel free to ask any math-related question, and I'll be happy to help you with it."

### Streaming

In [52]:
import time

In [58]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    time.sleep(0.02)
    print(r.content, end="")
    # print(r.content, end="|")

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


Hi Todd! Here's another joke for you: Why couldn't the bicycle stand up by itself? Because it was two tired!